In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.collections import LineCollection
import fastf1
from fastf1 import plotting
from scipy.interpolate import interp1d

In [ ]:
fastf1.Cache.enable_cache('./f1_cache')
fastf1.Cache.get_cache_info()

In [ ]:
session = fastf1.get_session(2025, 13, 'Sprint Qualifying')
session.load()

In [ ]:
pia_color = plotting.get_driver_color(session=session,identifier='PIA')
ver_color = plotting.get_driver_color(session=session,identifier='VER')

In [ ]:
ver_df = session.laps.pick_drivers('VER').pick_fastest().telemetry[['Time','Distance','X','Y']]
pia_df = session.laps.pick_drivers('PIA').pick_fastest().telemetry[['Time','Distance','X','Y']]

In [ ]:
ver_df['Seconds'] = ver_df['Time'].dt.total_seconds()
pia_df['Seconds'] = pia_df['Time'].dt.total_seconds()

In [ ]:
# Verstappen
ver_time_interp = interp1d(ver_df['Distance'], ver_df['Seconds'], kind='linear', bounds_error=False, fill_value="extrapolate")
ver_x_interp    = interp1d(ver_df['Distance'], ver_df['X'], kind='linear', bounds_error=False, fill_value="extrapolate")
ver_y_interp    = interp1d(ver_df['Distance'], ver_df['Y'], kind='linear', bounds_error=False, fill_value="extrapolate")

# Piastri
pia_time_interp = interp1d(pia_df['Distance'], pia_df['Seconds'], kind='linear', bounds_error=False, fill_value="extrapolate")
pia_x_interp    = interp1d(pia_df['Distance'], pia_df['X'], kind='linear', bounds_error=False, fill_value="extrapolate")
pia_y_interp    = interp1d(pia_df['Distance'], pia_df['Y'], kind='linear', bounds_error=False, fill_value="extrapolate")

In [ ]:
min_dist = max(pia_df['Distance'].min(), ver_df['Distance'].min())
max_dist = min(pia_df['Distance'].max(), ver_df['Distance'].max())

distance_grid = np.linspace(min_dist, max_dist, num=7000)  # adjust num=3000 for resolution

In [ ]:
# Interpolated cumulative time
ver_time = ver_time_interp(distance_grid)
pia_time = pia_time_interp(distance_grid)

# Interpolated positions
y = (ver_x_interp(distance_grid) + pia_x_interp(distance_grid)) / 2
x = -(ver_y_interp(distance_grid) + pia_y_interp(distance_grid)) / 2

In [ ]:
delta = pia_time - ver_time  # Negative -> Piastri faster

# Assign colors
colors = np.where(delta < 0, pia_color, ver_color)

In [ ]:
# Create line segments
points = np.array([x, y]).T.reshape(-1, 1, 2)
segments = np.concatenate([points[:-1], points[1:]], axis=1)

In [ ]:
import matplotlib.patches as mpatches

# Create legend handles manually
pia_patch = mpatches.Patch(color=pia_color, label='Piastri faster')
ver_patch = mpatches.Patch(color=ver_color, label='Verstappen faster')

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
fig.set_facecolor('#333333')
ax.axis('off')

# Color per segment (drop last color to match segments length)
lc = LineCollection(segments, colors=colors[:-1], linewidth=8)

ax.add_collection(lc)
ax.autoscale()

# Add legend to your axes
ax.legend(handles=[pia_patch, ver_patch],
          loc='upper left',  # or any location you prefer
          bbox_to_anchor=(0.04,1),
          frameon=True,
          facecolor='k',
          fontsize=20,
          labelcolor='white')

plt.tight_layout()
plt.show()